In [1]:
notes = ''

from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

# ------------------------------------------ #

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

def evolve_t(folder, nx, dt, T, degree, u_code, f_code, sigma, mu, velocity):
    t = 0.0
    u_exact = Expression(u_code, degree = degree+1, t = t)
    f = Expression(f_code, degree = degree+1, t = t)

#     mesh = UnitSquareMesh(nx,nx)
    mesh = RectangleMesh(Point(-2.0,-2.0), Point(2.0,2.0), 2*nx, 2*nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree+1, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n0 = interpolate(u_D, Q)
    u_n1 = Function(Q)
    u_ = Function(Q)

        # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)
    
# -------need u_n1------------------------------- #

    # Galerkin variational problem
    F = (u - u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)
    
#     # SUPG stabilization terms
#     h = CellDiameter(mesh)

#     # based on paper's definition of residual and stabilization term
#     Lt = -mu*div(grad(u)) + dot(velocity, grad(u)) + (sigma+1.0/dt)*u 
#     ft = u_n0/dt + f
#     r = ft - Lt
#     vnorm = sqrt(dot(velocity, velocity))
#     F -= dt*(h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx    
    
    
    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    t += dt

    # Assemble matrices
    A1 = assemble(a1)

    u_.rename('u','u')
    
    u_D.t = t
    f.t = t
    u_exact.t = t
    velocity.t = t

    b = assemble(L)
    bc = DirichletBC(Q, u_D, boundary)
    bc.apply(A1)
    bc.apply(b)
    
    solve(A1, u_n1.vector(), b)#, 'gmres')
    progress.update(t / T)

    
    # -------------------------------------- #

    # Galerkin variational problem
    F = (1.5*u - 2.0*u_n1 + 0.5*u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)
    
#     # based on paper's definition of residual and stabilization term
#     Lt = -mu*div(grad(u)) + dot(velocity, grad(u)) + (sigma+1.5/dt)*u 
#     ft = 2.0*u_n1/dt - 0.5*u_n0/dt + f
#     r = ft - Lt
#     vnorm = sqrt(dot(velocity, velocity))
#     F -= dt*(h/(2.0*vnorm))*dot(velocity, grad(v))*r*dx    

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)


    
    # Outputting files
    out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    out_file_ubar = File(folder+"E_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    
    u_.rename('u','u')
    
    # Save t = 0.0
#     out_file_uexact << (ue, float(t))
#     out_file_ubar << (u_, float(t))

    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt
        
        u_.rename('u','u')
        u_exact.rename('u','u')
        
        u_D.t = t
        f.t = t
        u_exact.t = t
        velocity.t = t
        # Assemble matrices
        
        A1 = assemble(a1)
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b)#, 'gmres')
        progress.update(t / T)
        
        out_file_ubar << (u_, float(t)) 
        
        u_n0.assign(u_n1)
        u_n1.assign(u_)

    L2, H1 = compute_errors(u_exact, u_, t, mesh)
    maxval, minval = compute_extrema(u_, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
    return str(L2)+','+str(H1)+','+str(maxval)+','+str(minval)

# ------------------------------------------ #

from datetime import datetime, date

def run(nx,dt,sigma,mu):
    d_date = datetime.now()
    today = d_date.strftime('%Y-%m-%d')
    f = open(today+'_log_SUPG.txt',"a+")
    
    T = 3
    degree = 2

    folder = 'EvolveOnly/sigma'+str(sigma)+'_mu'+str(mu)
    velocity = Expression(('cos(t)','sin(t)'), degree = degree, t = 0)
    adr_f = Expression('exp(-(pow(x[0]-0.5,2)+pow(x[1]-0.5,2))/pow(0.07,2))', degree = degree)


    sim_data = 'SUPG, dt='+str(dt)+', T='+str(T)+', P'+str(degree)
    f.write(sim_data+'\n')


    now = d_date.strftime("%I:%M:%S %p")
    f.write('start: '+now+'\n\n')

    output = evolve_t(folder, nx, dt, T, degree, '0', adr_f.cppcode, sigma, mu, velocity)
    f.write(str(nx)+','+str(dt)+','+output+'\n')


    d_date = datetime.now()
    now = d_date.strftime("%I:%M:%S %p")
    f.write('\nend: '+now+'\n\n')
    f.write(notes)

    f.close()

In [2]:
dt=0.1
sigma = 0.1
mu = 0.00005
# run(25,dt,sigma,mu)
# run(50,dt,sigma,mu)
run(100,dt,sigma,mu)
run(200,dt,sigma,mu)
run(400,dt,sigma,mu)

100 , 6.80546414688e+37 , 1.05694215724e+40 , 2.24756200886e+39 , -3.004112636e+39
200 , 2.13897603358e+15 , 6.3180577019e+17 , 1.34489520585e+17 , -4.26396927658e+16
400 , 0.0264024630596 , 0.356934075414 , 0.121577251898 , -0.0112647851648


In [3]:
run(600,dt,sigma,mu)

RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to successfully call PETSc function 'KSPSolve'.
*** Reason:  PETSc error code is: 76 (Error in external library).
*** Where:   This error was encountered inside /feedstock_root/build_artefacts/fenics_1514906081531/work/dolfin-2017.2.0.post0/dolfin/la/PETScKrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2017.2.0
*** Git changeset:  eff8a933c8ffe6b1204735d58ff25c5b75741fbe
*** -------------------------------------------------------------------------


In [4]:
dt=0.1
sigma = 0.1
mu = 0.0005
# run(25,dt,sigma,mu)
# run(50,dt,sigma,mu)
run(100,dt,sigma,mu)
run(200,dt,sigma,mu)
# run(400,dt,sigma,mu)

100 , 0.0258661422324 , 0.339183121216 , 0.118770355478 , -0.00983448923713
200 , 0.0258661544779 , 0.339179209059 , 0.118758356417 , -0.00983872503865


In [5]:
run(25,dt,sigma,mu)
run(50,dt,sigma,mu)

25 , 0.0256459972692 , 0.339533534429 , 0.119083673726 , -0.0114309666584
50 , 0.02586580471 , 0.339249708608 , 0.118974844965 , -0.00985827268232


In [6]:
run(400,dt,sigma,mu)

400 , 0.0258661546169 , 0.33917911525 , 0.118765712548 , -0.00984127817874


In [7]:
dt=0.1
sigma = 0.1
mu = 0.00005
run(25,dt,sigma,mu)
run(50,dt,sigma,mu)

25 , 1.24722202843e+15 , 2.01873752953e+16 , 5.4998133295e+14 , -1.89524619123e+16
50 , 1.56239861534e+34 , 3.08919281669e+35 , 3.44986797588e+33 , -2.94920480299e+35
